In [2]:
import pandas as pd
from sklearn.utils import shuffle
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
import pickle

In [3]:
import bikefind

In [4]:
from bikefind.linearRegression import getMergeTable, getForecastTable

In [5]:
df_merge = getMergeTable()

In [7]:
df_merge.head(10)

,time_x,address,totalBikeStands,availableBikeStands,availableBikes,status,datetime_x,date,day,hour,...,currentTemp,maxTemp,minTemp,pressure,humidity,windSpeed,windAngle,cloudDensity,visibility,datetime_y
0,1521225890000,Newman House,40,37,3,OPEN,2018-03-16 18:44:50,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
1,1521225890000,Western Way,40,39,1,CLOSED,2018-03-16 18:44:50,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
2,1521225891000,York Street West,40,33,7,OPEN,2018-03-16 18:44:51,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
3,1521225909000,Mount Brown,22,12,10,OPEN,2018-03-16 18:45:09,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
4,1521225951000,Brookfield Road,30,12,18,OPEN,2018-03-16 18:45:51,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
5,1521225972000,Eccles Street East,27,26,1,OPEN,2018-03-16 18:46:12,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
6,1521225982000,Fitzwilliam Square East,40,0,0,CLOSED,2018-03-16 18:46:22,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
7,1521225985000,Collins Barracks Museum,38,19,19,OPEN,2018-03-16 18:46:25,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
8,1521225995000,Merrion Square West,20,20,0,CLOSED,2018-03-16 18:46:35,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00
9,1521226003000,Benson Street,40,0,0,CLOSED,2018-03-16 18:46:43,2018-03-16,Friday,18,...,279.71,280.15,279.15,1000,93,6.7,110,40,10000,2018-03-16 18:30:00


In [8]:
df_merge.dtypes

time_x                          int64
address                        object
totalBikeStands                 int64
availableBikeStands             int64
availableBikes                  int64
status                         object
datetime_x             datetime64[ns]
date                           object
day                            object
hour                            int64
time_y                          int64
mainDescription                object
detailedDescription            object
icon                           object
currentTemp                   float64
maxTemp                       float64
minTemp                       float64
pressure                        int64
humidity                        int64
windSpeed                     float64
windAngle                       int64
cloudDensity                    int64
visibility                      int64
datetime_y             datetime64[ns]
dtype: object

In [9]:
df_merge['hour'] = df_merge['hour'].astype('category')
df_merge.dtypes

time_x                          int64
address                        object
totalBikeStands                 int64
availableBikeStands             int64
availableBikes                  int64
status                         object
datetime_x             datetime64[ns]
date                           object
day                            object
hour                         category
time_y                          int64
mainDescription                object
detailedDescription            object
icon                           object
currentTemp                   float64
maxTemp                       float64
minTemp                       float64
pressure                        int64
humidity                        int64
windSpeed                     float64
windAngle                       int64
cloudDensity                    int64
visibility                      int64
datetime_y             datetime64[ns]
dtype: object

In [10]:
for column in ['time_x', 'datetime_x', 'time_y', 'datetime_y', 'visibility', 'icon']:
    del df_merge[column]
df_merge.head(10)

,address,totalBikeStands,availableBikeStands,availableBikes,status,date,day,hour,mainDescription,detailedDescription,currentTemp,maxTemp,minTemp,pressure,humidity,windSpeed,windAngle,cloudDensity
0,Newman House,40,37,3,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
1,Western Way,40,39,1,CLOSED,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
2,York Street West,40,33,7,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
3,Mount Brown,22,12,10,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
4,Brookfield Road,30,12,18,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
5,Eccles Street East,27,26,1,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
6,Fitzwilliam Square East,40,0,0,CLOSED,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
7,Collins Barracks Museum,38,19,19,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
8,Merrion Square West,20,20,0,CLOSED,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
9,Benson Street,40,0,0,CLOSED,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40


In [11]:
df_merge.shape

(670341, 18)

In [12]:
df_merge.sort_values(by='date')

,address,totalBikeStands,availableBikeStands,availableBikes,status,date,day,hour,mainDescription,detailedDescription,currentTemp,maxTemp,minTemp,pressure,humidity,windSpeed,windAngle,cloudDensity
0,Newman House,40,37,3,OPEN,2018-03-16,Friday,18,Clouds,scattered clouds,279.71,280.15,279.15,1000,93,6.7,110,40
4962,Jervis Street,21,19,2,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1004,87,6.7,80,75
4961,Jervis Street,21,19,2,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1003,87,7.2,80,75
4960,Portobello Harbour,30,8,22,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1004,87,6.7,80,75
4959,Portobello Harbour,30,8,22,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1003,87,7.2,80,75
4958,Parnell Street,20,12,8,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1004,87,6.7,80,75
4957,Parnell Street,20,12,8,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1003,87,7.2,80,75
4956,Grand Canal Dock,40,40,0,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1004,87,6.7,80,75
4955,Grand Canal Dock,40,40,0,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1003,87,7.2,80,75
4954,Princes Street / O'Connell Street,23,0,23,OPEN,2018-03-16,Friday,22,Clouds,broken clouds,280.15,280.15,280.15,1004,87,6.7,80,75


In [13]:
df_merge.groupby('mainDescription').count().sort_values(by='address', ascending=False)

,address,totalBikeStands,availableBikeStands,availableBikes,status,date,day,hour,detailedDescription,currentTemp,maxTemp,minTemp,pressure,humidity,windSpeed,windAngle,cloudDensity
mainDescription,,,,,,,,,,,,,,,,,
Clouds,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602,374602
Rain,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881,157881
Drizzle,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356,44356
Snow,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704,30704
Mist,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025
Fog,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851,19851
Clear,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922,17922


In [14]:
print("All the weather that has happened in the last four weeks")
df_merge.groupby('detailedDescription').count().sort_values(by='address', ascending=False)

All the weather that has happened in the last four weeks


,address,totalBikeStands,availableBikeStands,availableBikes,status,date,day,hour,mainDescription,currentTemp,maxTemp,minTemp,pressure,humidity,windSpeed,windAngle,cloudDensity
detailedDescription,,,,,,,,,,,,,,,,,
broken clouds,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838,259838
light rain,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657,83657
scattered clouds,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882,76882
shower rain,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297,43297
few clouds,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801,36801
mist,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025,25025
light intensity shower rain,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124,23124
light intensity drizzle rain,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402,22402
light intensity drizzle,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954,21954


In [15]:
target = 'availableBikes'
descriptive = [ 'day', 'hour', 'address', 'mainDescription', 'detailedDescription', 'currentTemp', 'pressure', 'humidity', 'windSpeed', 'windAngle', 'cloudDensity', 'Intercept']

In [16]:
df_merge = shuffle(df_merge)

In [17]:
df_merge.head(10)

,address,totalBikeStands,availableBikeStands,availableBikes,status,date,day,hour,mainDescription,detailedDescription,currentTemp,maxTemp,minTemp,pressure,humidity,windSpeed,windAngle,cloudDensity
386273,Hatch Street,36,34,2,OPEN,2018-03-30,Friday,5,Clouds,broken clouds,275.73,276.15,275.15,995,100,1.0,0,75
376379,Leinster Street South,30,21,9,OPEN,2018-03-29,Thursday,21,Rain,moderate rain,277.15,277.15,277.15,995,93,4.1,80,75
474885,Parnell Square North,20,20,0,OPEN,2018-04-03,Tuesday,19,Clouds,broken clouds,282.15,282.15,282.15,988,70,3.1,190,75
393540,Dame Street,16,2,14,OPEN,2018-03-30,Friday,9,Clouds,broken clouds,278.15,278.15,278.15,996,93,3.6,40,75
272633,King Street North,30,0,30,OPEN,2018-03-26,Monday,0,Clouds,broken clouds,278.66,279.15,278.15,1016,86,3.1,260,75
90178,Heuston Station (Car Park),40,17,23,OPEN,2018-03-19,Monday,16,Clouds,scattered clouds,278.15,278.15,278.15,1024,60,9.3,40,40
132185,Fownes Street Upper,30,15,15,OPEN,2018-03-22,Thursday,9,Clouds,broken clouds,281.15,281.15,281.15,1016,65,7.2,210,75
523583,Sir Patrick's Dun,40,29,11,OPEN,2018-04-07,Saturday,19,Rain,light intensity shower rain,282.15,282.15,282.15,1006,87,2.1,130,40
402325,James Street,40,24,16,OPEN,2018-03-30,Friday,14,Rain,light intensity shower rain,277.39,279.15,276.15,997,87,7.2,10,75
300973,Wolfe Tone Street,29,25,4,OPEN,2018-03-26,Monday,19,Rain,light rain,281.73,282.15,281.15,1009,87,6.2,150,75


In [18]:
df_merge['Intercept'] = 1.0

In [ ]:
limit = (df_merge.shape[0] * 7) // 10
limit

In [ ]:
df_train = df_merge[:limit]
df_test = df_merge[limit:]

In [ ]:
X_train = df_train[descriptive]

y_train = df_train[target]

In [ ]:
X_train = pd.get_dummies(X_train)

In [ ]:
lm = LinearRegression().fit(X_train, y_train)

In [ ]:
features = list(X_train)
coefficients = lm.coef_
coefficient_strengths = pd.DataFrame({"Feature": features, "Coefficient":coefficients})
coefficient_strengths.set_index("Feature").sort_values(by="Coefficient", ascending = False)

In [ ]:
predictions_train = lm.predict(X_train)
print("Explained Variance:", metrics.explained_variance_score(y_train, predictions_train))
print("Mean Absolute Error:", metrics.mean_absolute_error(y_train, predictions_train))
print("R2 Score:", metrics.r2_score(y_train, predictions_train))

In [ ]:
def evaluator(X_train, X_test, y_train, y_test):
    lm = LinearRegression().fit(X_train, y_train)
    predictions = lm.predict(X_test)
    print("Explained Variance:", metrics.explained_variance_score(y_test, predictions))
    print("Mean Absolute Error:", metrics.mean_absolute_error(y_test, predictions))
    print("R2 Score:", metrics.r2_score(y_test, predictions))

In [ ]:
evaluator(X_train, X_train, y_train, y_train)

In [ ]:
print(descriptive)

In [ ]:
descriptive_min = ['day', 'hour', 'address', 'Intercept']

In [ ]:
X_min_train = df_train[descriptive_min]
X_min_train = pd.get_dummies(X_min_train)

In [ ]:
evaluator(X_min_train, X_min_train, y_train, y_train)

In [ ]:
descriptive_a = descriptive_min + ['mainDescription']
X_a_train = df_train[descriptive_a]
X_a_train = pd.get_dummies(X_a_train)

In [ ]:
evaluator(X_a_train, X_a_train, y_train, y_train)

In [ ]:
descriptive_b = descriptive_min + ['detailedDescription']
X_b_train = df_train[descriptive_b]
X_b_train = pd.get_dummies(X_b_train)

In [ ]:
evaluator(X_b_train, X_b_train, y_train, y_train)

In [ ]:
descriptive_c = descriptive_min + ['currentTemp', 'pressure', 'humidity', 'windSpeed', 'windAngle', 'cloudDensity']
X_c_train = df_train[descriptive_c]
X_c_train = pd.get_dummies(X_c_train)

In [ ]:
evaluator(X_c_train, X_c_train, y_train, y_train)

In [ ]:
descriptive_d = descriptive_c + ['mainDescription']
X_d_train = df_train[descriptive_d]
X_d_train = pd.get_dummies(X_d_train)

In [ ]:
evaluator(X_d_train, X_d_train, y_train, y_train)

In [ ]:
descriptive_e = ['currentTemp', 'pressure', 'humidity', 'windSpeed', 'windAngle', 'cloudDensity']
X_e_train = df_train[descriptive_e]
X_e_train = pd.get_dummies(X_e_train)

In [ ]:
evaluator(X_e_train, X_e_train, y_train, y_train)

In [ ]:
lm = LinearRegression().fit(X_e_train, y_train)
features = list(X_e_train)
coefficients = lm.coef_
coefficient_strengths = pd.DataFrame({"Feature": features, "Coefficient":coefficients})
coefficient_strengths.set_index("Feature").sort_values(by="Coefficient", ascending = False)

In [ ]:
descriptive_f = descriptive_c + ['detailedDescription']
X_f_train = df_train[descriptive_f]
X_f_train = pd.get_dummies(X_f_train)

In [ ]:
evaluator(X_f_train, X_f_train, y_train, y_train)

In [ ]:
descriptive_g = descriptive_min + ['mainDescription', 'detailedDescription']
X_g_train = df_train[descriptive_g]
X_g_train = pd.get_dummies(X_g_train)

In [ ]:
evaluator(X_g_train, X_g_train, y_train, y_train)

In [ ]:
descriptive_1 = descriptive_a + ['currentTemp']
X_1_train = df_train[descriptive_1]
X_1_train = pd.get_dummies(X_1_train)


In [ ]:
evaluator(X_1_train, X_1_train, y_train, y_train)

In [ ]:
descriptive_2 = descriptive_1 + ['windSpeed']
X_2_train = df_train[descriptive_2]
X_2_train = pd.get_dummies(X_2_train)

In [ ]:
evaluator(X_2_train, X_2_train, y_train, y_train)

In [ ]:
descriptive_3 = descriptive_2 + ['cloudDensity']
X_3_train = df_train[descriptive_3]
X_3_train = pd.get_dummies(X_3_train)

In [ ]:
evaluator(X_3_train, X_3_train, y_train, y_train)

In [ ]:
descriptive_3a = descriptive_1 + ['cloudDensity']
X_3a_train = df_train[descriptive_3a]
X_3a_train = pd.get_dummies(X_3a_train)

In [ ]:
evaluator(X_3a_train, X_3a_train, y_train, y_train)

In [ ]:
descriptive_4a = descriptive_3a + ['humidity']
X_4a_train = df_train[descriptive_4a]
X_4a_train = pd.get_dummies(X_4a_train)

In [ ]:
evaluator(X_4a_train, X_4a_train, y_train, y_train)

In [ ]:
descriptive_5a = descriptive_4a + ['pressure']
X_5a_train = df_train[descriptive_5a]
X_5a_train = pd.get_dummies(X_5a_train)

In [ ]:
evaluator(X_5a_train, X_5a_train, y_train, y_train)

In [ ]:
descriptive_6a = descriptive_5a + ['windAngle']
X_6a_train = df_train[descriptive_6a]
X_6a_train = pd.get_dummies(X_6a_train)

In [ ]:
evaluator(X_6a_train, X_6a_train, y_train, y_train)

Best model is still the original one, taking in all available features.

In [ ]:
X_train = df_train[descriptive]
y_train = df_train[target]
X_train = pd.get_dummies(X_train)

In [ ]:
evaluator(X_train, X_train, y_train, y_train)

In [ ]:
X_test = df_test[descriptive]
y_test = df_test[target]
X_test = pd.get_dummies(X_test)

In [ ]:
evaluator(X_train, X_test, y_train, y_test)

In [ ]:
lm = LinearRegression().fit(X_train, y_train)

In [23]:
X = df_merge[descriptive]
y = df_merge[target]
X = pd.get_dummies(X)
X = X.reindex_axis(sorted(X.columns), axis=1)
lm = LinearRegression().fit(X, y)

/home/eoin/anaconda3/envs/Software/lib/python3.6/site-packages/ipykernel/__main__.py:4: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.


In [24]:
print(list(X))

['Intercept', 'address_Barrow Street', 'address_Benson Street', 'address_Blackhall Place', 'address_Blessington Street', 'address_Bolton Street', 'address_Brookfield Road', 'address_Cathal Brugha Street', 'address_Charlemont Street', 'address_Christchurch Place', 'address_City Quay', 'address_Clarendon Row', 'address_Clonmel Street', 'address_Collins Barracks Museum', 'address_Convention Centre', 'address_Custom House', 'address_Custom House Quay', 'address_Dame Street', 'address_Denmark Street Great', 'address_Deverell Place', 'address_Earlsfort Terrace', 'address_Eccles Street', 'address_Eccles Street East', 'address_Emmet Road', 'address_Exchequer Street', 'address_Excise Walk', 'address_Fenian Street', 'address_Fitzwilliam Square East', 'address_Fitzwilliam Square West', 'address_Fownes Street Upper', 'address_Francis Street', 'address_Frederick Street South', "address_George's Lane", 'address_Georges Quay', 'address_Golden Lane', 'address_Grand Canal Dock', 'address_Grangegorman L

In [25]:
pickle.dump(lm, open( "model.p", "wb" ) )

In [ ]:
features = list(X)

In [ ]:
pickle.dump(features, open("features.p", "wb"))

In [ ]:
comparison = pd.DataFrame({"Actual":y, "Predicted":lm.predict(X)})

In [ ]:
comparison

In [ ]:
lm.coef_

In [ ]:
df_merge["Predicted"] = lm.predict(X)

In [ ]:
df_merge.iloc[0]